In [1]:
from langchain_groq import ChatGroq

In [17]:
llm=ChatGroq (
temperature=0,
groq_api_key='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',
model_name="llama3-70b-8192"
)
response = llm.invoke("The first person to land on moon was ..")
print(response.content)

That's an easy one!

The first person to set foot on the Moon was Neil Armstrong. He stepped out of the lunar module Eagle and onto the Moon's surface on July 20, 1969, during the Apollo 11 mission. Armstrong famously declared, "That's one small step for man, one giant leap for mankind," as he became the first human to walk on the Moon.


In [20]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://careers.nike.com/3d-footwear-designer-ii/job/R-60083")
page_data = loader.load().pop().page_content
print (page_data)






















3D Footwear Designer II










































Skip to main content
Open Virtual Assistant










Home


Career Areas


Total Rewards


Life@Nike


Purpose










Language





Select a Language

  Deutsch  
  English  
  Español (España)  
  Español (América Latina)  
  Français  
  Italiano  
  Nederlands  
  Polski  
  Tiếng Việt  
  Türkçe  
  简体中文  
  繁體中文  
  עִברִית  
  한국어  
  日本語  








Careers


















Close Menu







Careers






Chat






                                Home
                            



                                Career Areas
                            



                                Total Rewards
                            



                                Life@Nike
                            



                                Purpose
                            










Jordan Careers







Converse Careers










Language











Menu



Return to Previous Menu



Se

In [21]:
from langchain_core.prompts import PromptTemplate
prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE:
    {page_data}
    ### INSTRUCTION:
    The scraped text is from the career's page of a website.
    Your job is to extract the job postings and return them in JSON format containing the
    following keys: `role`, `experience`, `skills and description` .
    Only return the valid JSON.
    ### VALID JSON (NO PREAMBLE):
    """
)
chain_extract = prompt_extract | llm
res=chain_extract.invoke(input={'page_data': page_data})
print(res.content)

Here is the extracted job posting in JSON format:

```
[
  {
    "role": "3D Footwear Designer II",
    "experience": "2-3 years",
    "skills": [
      "3D Design techniques",
      "emerging tools and technologies",
      "knowledge of product creation",
      "polygonal and surface modeling",
      "digital design applications (i.e. Maya, Blender, Rhino, ZBrush, Blender, Photoshop, and Illustrator)",
      "AI (Artificial Intelligence) tools",
      "crafting conceptual prototypes",
      "preparing 3D assets for rapid prototyping (RP)"
    ],
    "description": "As a 3D Footwear Designer, you will be working under the Digital Product Creation Organization while focusing on 3D Design for Global Footwear. You will also collaborate with other members of the design team including product, materials, color, computational designers as well as Development and Marketing as part of a creative collective on each project."
  }
]
```


In [25]:
#type str to json(dict) 
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

[{'role': '3D Footwear Designer II',
  'experience': '2-3 years',
  'skills': ['3D Design techniques',
   'emerging tools and technologies',
   'knowledge of product creation',
   'polygonal and surface modeling',
   'digital design applications (i.e. Maya, Blender, Rhino, ZBrush, Blender, Photoshop, and Illustrator)',
   'AI (Artificial Intelligence) tools',
   'crafting conceptual prototypes',
   'preparing 3D assets for rapid prototyping (RP)'],
  'description': 'As a 3D Footwear Designer, you will be working under the Digital Product Creation Organization while focusing on 3D Design for Global Footwear. You will also collaborate with other members of the design team including product, materials, color, computational designers as well as Development and Marketing as part of a creative collective on each project.'}]

In [26]:
import pandas as pd
df=pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [31]:
import uuid
import chromadb
client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")
if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas = {"links": row["Links"]},
                                  ids=[str(uuid.uuid4())])

In [33]:
links = collection.query(query_texts=["Experience in Python", "Expertise in React Native"],n_results=2).get('metadatas')
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/python-portfolio'}],
 [{'links': 'https://example.com/react-native-portfolio'},
  {'links': 'https://example.com/react-portfolio'}]]

In [36]:
job = json_res[0]  
print(job['skills']) 

['3D Design techniques', 'emerging tools and technologies', 'knowledge of product creation', 'polygonal and surface modeling', 'digital design applications (i.e. Maya, Blender, Rhino, ZBrush, Blender, Photoshop, and Illustrator)', 'AI (Artificial Intelligence) tools', 'crafting conceptual prototypes', 'preparing 3D assets for rapid prototyping (RP)']


In [39]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
    the seamless integration of business processes through automated tools.
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
    process optimization, cost reduction, and heightened overall efficiency.
    Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
    in fulfilling their needs.
    Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
    Remember you are Mohan, BDE at AtliQ.
    Do not provide a preamble.
    ### EMAIL (NO PREAMBLE):
    """
)
#{link_list} prompt template argument 
chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert 3D Footwear Design Solutions for Your Organization

Dear Hiring Manager,

I came across the job posting for a 3D Footwear Designer II and was impressed by the emphasis on leveraging emerging tools and technologies to drive innovation in footwear design. As a business development executive at AtliQ, I believe our expertise in AI-powered solutions can significantly enhance your design process.

AtliQ has a proven track record of empowering enterprises with tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team of experts can help you streamline your 3D design workflow, leveraging AI tools to accelerate conceptual prototyping, polygonal and surface modeling, and preparing 3D assets for rapid prototyping.

Our portfolio showcases our capabilities in AI and software consulting, including:

* AI-powered Python solutions: https://example.com/ml-python-portfolio
* React-based design applications: https://exa